In [212]:
import calendar
import datetime
import ee
import geemap
import shapely

import geopandas as gpd
import numpy as np
import pandas as pd

from pathlib import Path
from typing import Sequence
from ursa_backend.code.common import bbox_to_ee

In [3]:
ee.Initialize(project="ee-ursa-test")

In [50]:
df = gpd.read_file(r"C:\Users\lain\OneDrive - Instituto Tecnologico y de Estudios Superiores de Monterrey\population_grids_data\final\reprojected\merged\02.2.02.gpkg")

xmin, ymin, xmax, ymax = df.total_bounds
xmin -= 5000
ymin -= 5000
xmax += 5000
ymax += 5000

bbox = gpd.GeoSeries([shapely.box(xmin, ymin, xmax, ymax)], crs=df.crs).to_crs("EPSG:4326").item()
bbox = bbox_to_ee(bbox)

In [123]:
def get_date_ranges(dates: Sequence[int], interval: datetime.timedelta):
    dates = sorted(dates)
    dates = pd.to_datetime(dates, unit="ms")

    date_ranges = []
    while True:
        start_date = dates[0]
        end_date = start_date + interval
        mask = (start_date <= dates) & (dates < end_date)
        all_dates = dates[mask].astype(int).to_numpy() // 10**6
        date_ranges.append((int(all_dates.min()), int(all_dates.max())))
        
        dates = dates[~mask]
        if len(dates) == 0:
            break
        
    return date_ranges

In [209]:
col = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterBounds(bbox)
)

temp_col = col.filterDate("2023-01-01", "2023-12-31")
dates = sorted(temp_col.aggregate_array("system:time_start").getInfo())
interval = datetime.timedelta(days=1)
date_ranges = get_date_ranges(dates, interval)

for start_date, end_date in date_ranges[15:]:
    print(pd.to_datetime((start_date, end_date), unit="ms"))
    filtered_col = col.filterDate(start_date, end_date + 1)
    print(filtered_col.size().getInfo())
    break

DatetimeIndex(['2023-03-25 18:10:37.781000', '2023-03-25 18:11:01.676000'], dtype='datetime64[ns]', freq=None)
2


In [213]:
calendar.monthrange(2023, 1)

(calendar.SUNDAY, 31)

In [218]:
f"{year}-{month_str}-01", f"{year}-{month_str}-{end_day}"

('2023-01-01', '2023-01-(calendar.SUNDAY, 31)')

In [ ]:
year = 2023
month = 1
month_str = str(month).rjust(2, "0")
_, end_day = calendar.monthrange(year, month)
end_day_str = str(end_day).rjust(2, "0")
start = f"{year}-{month_str}-01"
end = f"{year}-{month_str}-{end_day_str}"

col = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterBounds(bbox)
    .filterDate(f"")
)

test = col.mean().clip(bbox)
m = geemap.Map(center=(31.8557, -116.6057), zoom=9, width=1400, height=800)
m.add_layer(test)
m

EEException: DateRange: Bad date/time '2023-01-32'.

In [ ]:
col = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterBounds(bbox)
)

In [5]:
import numpy as np

test = [1, 2, 3, 5, 5, 5, 5, 5, 5]
test[:np.argmax(test) + 1]

[1, 2, 3, 5]